In [1]:
from langchain_exa import ExaSearchRetriever

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="RAG/Web RAG/.env")

True

In [3]:
retreiver = ExaSearchRetriever(k=3, highlights=True)    # k for the number of webpages to return and highlights gives us some breife summary in the metadata when we print below
documents = retreiver.invoke("best time to visit japan")

In [4]:
print(documents[0].metadata["highlights"])
print(documents[0].metadata["url"])

['- [Do you have any recommendations for first-time visitors to Japan? ]<web_link> - [I would like to hire a personal guide. ]<web_link> - [Which tour operators specialise in Japan? ]<web_link> - [When is the best time to visit Japan? ]<web_link>']
https://faq.japan-travel.jnto.go.jp/en/faq/articles/102117


In [5]:
from langchain_exa import ExaSearchRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


retriever = ExaSearchRetriever(k=3, highlights=True)

document_prompt = PromptTemplate.from_template("""
<source>                                        
    <url>{url}</url>
    <highlights>{highlights}</highlights>
</source>
""")

document_chain = RunnableLambda(
    lambda document: {
        "highlights": document.metadata["highlights"], 
        "url": document.metadata["url"]
    }
) | document_prompt

retrieval_chain = retriever | document_chain.map() | (lambda docs: "\n".join([i.text for i in docs]))   # .map() is a langchain function that invokes a chain one element at a time

print(retrieval_chain.invoke("best time to visit japan"))


<source>                                        
    <url>https://faq.japan-travel.jnto.go.jp/en/faq/articles/102117</url>
    <highlights>['- [Do you have any recommendations for first-time visitors to Japan? ]<web_link> - [I would like to hire a personal guide. ]<web_link> - [Which tour operators specialise in Japan? ]<web_link> - [When is the best time to visit Japan? ]<web_link>']</highlights>
</source>


<source>                                        
    <url>https://www.japan.travel/en/blog/choosing-the-best-time-to-visit-japan/</url>
    <highlights>['- [Coronavirus (COVID-19) Advisory Information](https://www.japan.travel/en/news/coronavirus/) - Select Language简体中文繁體中文（香港）繁體中文（臺灣）India (English)Bahasa Indonesia한국어ภาษาไทยTiếng ViệtSingapore (English)Philippines (English)Malaysia (English)Australia/New Zealand (English)FrançaisDeutschItalianoEspañolUnited Kingdom (English)Nordic countries（English）Canada (English)Canada (Français)United States (English)Mexico (español)Português

In [6]:
Query_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert research assistant. You use xml-formatted context to research people's questions."),
    ("human", """
Please answer the following query based on the provided context. Please cite your sources at the end of your response.:
     
Query: {query}
--------------------
<context>
{retriver}
</context>
""")
])

llm = ChatOpenAI(temperature=0.2)

Main_chain = RunnableParallel({
    "query" : RunnablePassthrough(),
    "retriver": retrieval_chain,
})| Query_prompt | llm

In [7]:
Main_chain.invoke("best time to visit japan")

AIMessage(content='Based on the provided context, the best time to visit Japan is from late March to early April, and from September to November. During the spring, the country is covered in cherry blossom trees, while autumn offers beautiful foliage with shades of red, orange, and gold. The weather during these periods is mild and pleasant for exploring the sights. For immersing yourself in the pink tint of sakura season in major cities like Tokyo, Kyoto, and Osaka, the start of April is recommended.\n\nSources:\n1. Japan National Tourism Organization: [When is the best time to visit Japan?](https://faq.japan-travel.jnto.go.jp/en/faq/articles/102117)\n2. Japan Travel: [Choosing the Best Time to Visit Japan](https://www.japan.travel/en/blog/choosing-the-best-time-to-visit-japan/)\n3. Booking.com: [Best Time to Visit Japan](https://www.booking.com/best-time/country/jp.tl.html)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 201, 'prompt_tok